In [ ]:
pip install openpyxl

In [ ]:
# Scraper do Whoscored
# Alterar matchLink, roundNr e matchGame manualmente a cada jogo

import json
import numpy as np
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from mplsoccer import Pitch,VerticalPitch
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager
import math
import ast
from tqdm import tqdm

website = 'https://www.whoscored.com'
matchLink = "https://www.whoscored.com/Matches/1659267/Live/Portugal-Liga-Portugal-2022-2023-Gil-Vicente-Portimonense"
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path=r'/Users/andreviegassimoes/Documents/VS Code/chromedriver',options=options)
driver.get(matchLink)
matchId = re.search('Matches/(\d+)/Live', matchLink)
matchId = matchId.group(1)
roundNr = '12'
matchGame = 'Gil Vicente FC vs Portimonense SC'

soup = BeautifulSoup(driver.page_source,'lxml')
driver.quit()

data_cen = re.compile('var matchCentreData = ({.*?})')

data = soup.find_all("script")

event_data = str(data)

dicte = event_data.split("matchCentreData: ")
dicte = dicte[1].splitlines()
x = dicte[0][:-1]

matchdata = json.loads(x) 

x = pd.DataFrame.from_dict(matchdata["events"])
home = pd.DataFrame.from_dict(matchdata["home"],orient='index')
away = pd.DataFrame.from_dict(matchdata["away"],orient='index')

home[1] = away
print(home[0][0])
print(home[1][0])
home= home.T



In [ ]:
# Organização geral dos dados

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
to_drop =  ['satisfiedEventsTypes']
x.drop(to_drop, inplace=True, axis=1)

In [ ]:
# Whoscored (100x100) para metros (105x68)

In [ ]:
for row, id in enumerate(x.x):
    x.x[row] = x.x[row]*105/100
for row, id in enumerate(x.endX):
    x.endX[row] = x.endX[row]*105/100
for row, id in enumerate(x.blockedX):
    x.blockedX[row] = x.blockedX[row]*105/100
for row, id in enumerate(x.y):
    x.y[row] = x.y[row]*68/100
for row, id in enumerate(x.endY):
    x.endY[row] = x.endY[row]*68/100
for row, id in enumerate(x.blockedY):
    x.blockedY[row] = x.blockedY[row]*68/100


In [ ]:
x.insert(len(x.keys()), 'timestamp', "")
for row, id in enumerate(x.timestamp):
    x.timestamp[row] = x.minute[row]*60+x.second[row]

In [ ]:
x['timestamp'] = x['timestamp'].fillna(0).astype('int')

In [ ]:
# Match Id e Field
x.insert(1, 'gameWeek', "")
x.insert(2, 'match', "")
x.insert(3, 'matchId', "")
x.insert(4, 'homeTeamId', "")
x.insert(5, 'awayTeamId', "")
x['gameWeek'] = x['gameWeek'].fillna(0).astype('string')
x['match'] = x['match'].fillna(0).astype('string')

In [ ]:
for row, id in enumerate(x.gameWeek):
    x.gameWeek[row] = roundNr
for row, id in enumerate(x.match):
    x.match[row] = matchGame
for row, id in enumerate(x.matchId):
    x.matchId[row] = matchId

In [ ]:
# Organização de dados

x.period = x['period'].replace("{'value': 1, 'displayName': 'FirstHalf'}", '1st Half')
x.period = x['period'].replace("{'value': 2, 'displayName': 'SecondHalf'}", '2nd Half')
x.period = x['period'].replace("{'value': 14, 'displayName': 'PostGame'}", 'Post Game')
x.period = x['period'].replace("{'value': 16, 'displayName': 'PreMatch'}", 'Pre Match')
x.outcomeType = x['outcomeType'].replace("{'value': 1, 'displayName': 'Successful'}", 'Successful')
x.outcomeType = x['outcomeType'].replace("{'value': 0, 'displayName': 'Unsuccessful'}", 'Unsuccessful')
x.type = x['type'].replace("{'value': 32, 'displayName': 'Start'}", 'Start')
x.type = x['type'].replace("{'value': 1, 'displayName': 'Pass'}", 'Pass')
x.type = x['type'].replace("{'value': 44, 'displayName': 'Aerial'}", 'Aerial Duel')
x.type = x['type'].replace("{'value': 50, 'displayName': 'Dispossessed'}", 'Dispossessed')
x.type = x['type'].replace("{'value': 7, 'displayName': 'Tackle'}", 'Tackle')
x.type = x['type'].replace("{'value': 13, 'displayName': 'MissedShots'}", 'Missed Shot')
x.type = x['type'].replace("{'value': 30, 'displayName': 'End'}", 'End')
x.type = x['type'].replace("{'value': 34, 'displayName': 'FormationSet'}", 'Formation Set')
x.type = x['type'].replace("{'value': 8, 'displayName': 'Interception'}", 'Interception')
x.type = x['type'].replace("{'value': 49, 'displayName': 'BallRecovery'}", 'Ball Recovery')
x.type = x['type'].replace("{'value': 12, 'displayName': 'Clearance'}", 'Clearance')
x.type = x['type'].replace("{'value': 6, 'displayName': 'CornerAwarded'}", 'Corner Awarded')
x.type = x['type'].replace("{'value': 61, 'displayName': 'BallTouch'}", 'Ball Touch')
x.type = x['type'].replace("{'value': 74, 'displayName': 'BlockedPass'}", 'Blocked Pass')
x.type = x['type'].replace("{'value': 45, 'displayName': 'Challenge'}", 'Dribbled')
x.type = x['type'].replace("{'value': 3, 'displayName': 'TakeOn'}", 'Dribble')
x.type = x['type'].replace("{'value': 42, 'displayName': 'GoodSkill'}", 'Good Skill')
x.type = x['type'].replace("{'value': 52, 'displayName': 'KeeperPickup'}", 'Keeper Pickup')
x.type = x['type'].replace("{'value': 4, 'displayName': 'Foul'}", 'Foul')
x.type = x['type'].replace("{'value': 10000, 'displayName': 'OffsideGiven'}", 'Offside Given')
x.type = x['type'].replace("{'value': 2, 'displayName': 'OffsidePass'}", 'Offside Pass')
x.type = x['type'].replace("{'value': 55, 'displayName': 'OffsideProvoked'}", 'Offside Provoked')
x.type = x['type'].replace("{'value': 16, 'displayName': 'Goal'}", 'Goal')
x.type = x['type'].replace("{'value': 11, 'displayName': 'Claim'}", 'Claim')
x.type = x['type'].replace("{'value': 18, 'displayName': 'SubstitutionOff'}", 'Substitution Off')
x.type = x['type'].replace("{'value': 19, 'displayName': 'SubstitutionOn'}", 'Substitution On')
x.type = x['type'].replace("{'value': 40, 'displayName': 'FormationChange'}", 'Formation Change')
x.type = x['type'].replace("{'value': 17, 'displayName': 'Card'}", 'Card')
x.type = x['type'].replace("{'value': 59, 'displayName': 'KeeperSweeper'}", 'Keeper Sweeper')
x.type = x['type'].replace("{'value': 58, 'displayName': 'PenaltyFaced'}", 'Penalty Faced')
x.type = x['type'].replace("{'value': 15, 'displayName': 'SavedShot'}", 'Saved Shot')
x.type = x['type'].replace("{'value': 10, 'displayName': 'Save'}", 'Save')
x.type = x['type'].replace("{'value': 51, 'displayName': 'Error'}", 'Error')
x.type = x['type'].replace("{'value': 56, 'displayName': 'ShieldBallOpp'}", 'Shield Ball OOP')
x.type = x['type'].replace("{'value': 41, 'displayName': 'Punch'}", 'Punch')
x.type = x['type'].replace("{'value': 54, 'displayName': 'Smother'}", 'Smother')
x.type = x['type'].replace("{'value': 14, 'displayName': 'ShotOnPost'}", 'Shot On Post')
x.type = x['type'].replace("{'value': 60, 'displayName': 'ChanceMissed'}", 'Chance Missed')
x.cardType = x['cardType'].replace("{'value': 31, 'displayName': 'Yellow'}", 'Yellow Card')
x.cardType = x['cardType'].replace("{'value': 33, 'displayName': 'Red'}", 'Red Card')
x.type = x['type'].replace("{'value': 53, 'displayName': 'CrossNotClaimed'}", 'Cross Not Claimed')


In [ ]:
for index, type in enumerate(x.type):
    if (type == 'Dribbled'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Goal'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Shot On Post'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Missed Shot'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Save'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Offside Pass'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Clearance'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Interception'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Dispossessed'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Tackle') and (x.outcomeType[index] == 'Successful'):
        x.type[index]='Complete Tackle'
for index, type in enumerate(x.type):
    if (type == 'Tackle'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Complete Tackle'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Aerial Duel') and (x.outcomeType[index] == 'Unsuccessful'):
        x.type[index]='Aerial Duel Lost'
for index, type in enumerate(x.type):
    if (type == 'Aerial Duel') and (x.outcomeType[index] == 'Successful'):
        x.type[index]='Aerial Duel Won'
for index, type in enumerate(x.type):
    if (type == 'Aerial Duel Lost'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Aerial Duel Won'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Corner Awarded') and (x.outcomeType[index] == 'Unsuccessful'):
        x.type[index]='Corner Conceded'
for index, type in enumerate(x.type):
    if (type == 'Corner Awarded'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Corner Conceded'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Foul') and (x.outcomeType[index] == 'Successful'):
        x.type[index]='Fouled'
for index, type in enumerate(x.type):
    if (type == 'Foul'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Fouled'):
        x.outcomeType[index]=''

In [ ]:
x['playerId'] = x['playerId'].fillna(0).astype('int')
x['teamId'] = x['teamId'].fillna(0).astype('int')
x['minute'] = x['minute'].fillna(0).astype('int')
x['expandedMinute'] = x['expandedMinute'].fillna(0).astype('int')
x['second'] = x['second'].fillna(0).astype('int')
x['period'] = x['period'].fillna(0).astype('string')
x['type'] = x['type'].fillna(0).astype('string')
x['matchId'] = x['matchId'].fillna(0).astype('int')
x['outcomeType'] = x['outcomeType'].fillna(0).astype('string')
x['relatedEventId'] = x['relatedEventId'].fillna(0).astype('int')
x['relatedPlayerId'] = x['relatedPlayerId'].fillna(0).astype('int')
x['isGoal'] = x['isGoal'].fillna(0).astype('bool')
x['isShot'] = x['isShot'].fillna(0).astype('bool')
x['cardType'] = x['cardType'].fillna(0).astype('string')

In [ ]:
min_dribble_length: float = 0.0
max_dribble_length: float = 105.0
max_dribble_duration: float = 20.0
next_actions = x.shift(-1)
same_team = x.teamId == next_actions.teamId
dx = x.endX - next_actions.x
dy = x.endY- next_actions.y
far_enough=dy** 2 + dy** 2 >= min_dribble_length** 2
not_too_far = dx ** 2+ dy** 2<= max_dribble_length ** 2
dt = next_actions.timestamp - x.timestamp
same_phase = dt < max_dribble_duration
same_period = x.period == next_actions.period
dribble_idx = same_team & far_enough & not_too_far & same_phase & same_period
dribbles = pd.DataFrame()
prev = x[dribble_idx]
nex= next_actions[dribble_idx]
dribbles['matchId' ] = nex.matchId
dribbles['period' ] = nex.period

for cols in ['expandedMinute']:
	dribbles[cols] = nex[cols]
for cols in ['passKey','assist']:
	dribbles[ cols] = [True for _ in range (len (dribbles))]
	dribbles['isTouch'] = [True for _ in range (len (dribbles))]
	morecols = ['playerId']
for cols in morecols:
		dribbles[cols] = nex[cols]
dribbles['eventId'] = prev.eventId + 0.1
dribbles['id'] = prev.id + 0.1
dribbles['matchId'] = x['matchId']
dribbles['match'] = x['match']
dribbles['gameWeek'] = x['gameWeek']
dribbles['timestamp' ] = (prev.timestamp + nex.timestamp) / 2
dribbles['teamId'] = nex.teamId
dribbles['x'] = prev.endX
dribbles['y']= prev.endY
dribbles['endX'] = nex.x
dribbles['endY'] = nex.y
dribbles['type'] = ['Carry' for _ in range(len(dribbles))]
dribbles['outcomeType'] = ['Successful' for _ in range(len(dribbles))]
dribbles['qualifiers']=[{} for _ in range(len(dribbles))]


In [ ]:
dribbles.insert(17,'distance', "")

In [ ]:
dribbles['distance'] = dribbles['distance'].fillna(0).astype('bool')

In [ ]:
dribbles['distance'] = np.sqrt(np.square(dribbles.x - dribbles.endX) + np.square(dribbles.y - dribbles.endY))

In [ ]:
x=pd.concat([x,dribbles], join='outer',axis=0)
x=x.sort_values(['id','eventId', 'timestamp']).reset_index(drop=false)
x['eventId']=range(len(x))

In [ ]:
temp_x = home.drop_duplicates(subset='teamId')

home_tmp = temp_x.drop(['scores', 'players', 'stats', 'formations', 'incidentEvents', 'name', 'countryName', 'managerName', 'shotZones', 'averageAge'], axis=1)
x = x.merge(home_tmp, left_on=['teamId'], right_on=['teamId'], how='left', suffixes=['x', 'home'])
x_columns = [col for col in x.columns if col != 'field']
x_columns.insert(6, 'field')
x = x[x_columns]
for row, field in enumerate(x.field):
    if field == 'home':
        x.homeTeamId = x.teamId[row]
    elif field == 'away':
        x.awayTeamId = x.teamId[row]

In [ ]:
x.insert(len(x.keys()), 'isGoodSkill', "")
for row, type in enumerate(x.type):
    if (type == 'Good Skill'):
        if (x.type[row-1] == 'Dribble') and (x.outcomeType[row-1] == 'Successful'):
            x.isGoodSkill[row-1] = 'True'
x.insert(len(x.keys()), 'bodyPart', "")
x.insert(len(x.keys()), 'situation', "")
x.insert(len(x.keys()), 'passType', "")
x.insert(len(x.keys()), 'isChipped', "")
x.insert(len(x.keys()), 'isBigChance', "")
x.insert(len(x.keys()), 'isKeyPass', "")
x.insert(len(x.keys()), 'isAssist', "")
x.insert(len(x.keys()), 'isLongBall', "")
x['bodyPart'] = x['bodyPart'].fillna(0).astype('string')

x['situation'] = x['situation'].fillna(0).astype('string')
x['passType'] = x['passType'].fillna(0).astype('string')
x['isKeyPass'] = x['isKeyPass'].fillna(0).astype('bool')
x['isBigChance'] = x['isBigChance'].fillna(0).astype('bool')
x['isAssist'] = x['isAssist'].fillna(0).astype('bool')
x['isLongBall'] = x['isLongBall'].fillna(0).astype('bool')
x['isChipped'] = x['isChipped'].fillna(0).astype('bool')
for row, type in enumerate(x.type):
    if (type == 'Goal'):
        if (x.type[row-1] == 'Pass') and (x.outcomeType[row-1] == 'Successful'):
            x.isAssist[row-1] = 'True'

for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('Chipped') != -1):
        x.isChipped[index] = 'True'
for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('Longball') != -1):
        x.isLongBall[index] = 'True'

for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('RightFoot') != -1):
        x.bodyPart[index]='Right Foot'
    if (qualifier.find('LeftFoot') != -1):
        x.bodyPart[index]='Left Foot'
    if (qualifier.find('Head') != -1):
        x.bodyPart[index]='Head'
    if (qualifier.find('OtherBodyPart') != -1):
        x.bodyPart[index]='Other Body Part'
    if (qualifier.find('Hands') != -1):
        x.bodyPart[index]='Hands'
    if (qualifier.find('Feet') != -1):
        x.bodyPart[index]='Feet'
for index, qualifier in enumerate(x.qualifiers):

    
    if (qualifier.find('Cross') != -1):
        x.passType[index]='Cross'
    if (qualifier.find('Throughball') != -1):
        x.passType[index]='Through Ball'
    if (qualifier.find('Headpass') != -1):
        x.passType[index]='Head Pass'
    if (qualifier.find('CornerTaken') != -1):
        x.situation[index]='Corner'
    if (qualifier.find('FreekickTaken') != -1):
        x.situation[index]='Freekick'
    if (qualifier.find('ThrowIn') != -1):
        x.situation[index]='Throw In'
    if (qualifier.find('KeeperThrow') != -1):
        x.situation[index]='Keeper Throw'  

for index, type in enumerate(x.type):
    if (type == 'Pass') and ((x.endY[index]-x.y[index]) > 40.8) :
        x.passType[index]='Switch Of Play'
for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('BigChance') != -1):
        x.isBigChance[index]='True'
for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('KeyPass') != -1):
        x.isKeyPass[index]='True'
for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('RegularPlay') != -1):
        x.situation[index]='Regular Play'
    if (qualifier.find('FromCorner') != -1):
        x.situation[index]='From Corner'
    if (qualifier.find('DirectFreekick') != -1):
        x.situation[index]='Direct Freekick'
    if (qualifier.find('FastBreak') != -1):
        x.situation[index]='Fast Break'
    if (qualifier.find('FastBreak') != -1):
        x.situation[index]='Fast Break'
    if (qualifier.find('GoalKick') != -1):
        x.situation[index]='Goal Kick' 
    if (qualifier.find('IndirectFreekickTaken') != -1):
        x.situation[index]='Indirect Freekick'
    if (qualifier.find('SetPiece') != -1):
        x.situation[index]='Set Piece'
    if (qualifier.find('ThrowinSetPiece') != -1):
        x.situation[index]='From Throw In Set Piece'
    if (qualifier.find('Penalty') != -1):
        x.situation[index]='Penalty'    

In [ ]:
# xG e xGOT do Fotmob (excel extraído pelo R)
xG = pd.read_excel ('shotsTotal.xlsx')
xG2 = xG.drop_duplicates(subset='id')
xG=xG2.drop(['event_type','team_id', 'player_id','player_name', 'x','y', 'min', 'min_added','is_blocked','is_on_target','blocked_x','blocked_y','goal_crossed_y','goal_crossed_z','shot_type','situation','period','on_goal_shot','first_name','last_name','team_color'], axis=1)
x= x.merge(xG, left_on=['id'], right_on=['id'], how='left', suffixes=['x','x'])
x_columns = [col for col in x.columns if col != 'is_own_goal']
x_columns.insert(31, 'is_own_goal')
x = x[x_columns]
x.rename(columns={'is_own_goal': 'isOwnGoal'}, inplace=True)
x.rename(columns={'expected_goals': 'xG'}, inplace=True)
x.rename(columns={'expected_goals_on_target': 'xGOT'}, inplace=True)
x.insert(44,'path', '')

In [ ]:
x.insert(len(x.keys()), 'receivedFrom', "")
x.insert(len(x.keys()), 'deliveredTo', "")
for row, type in enumerate(x.type):
    if (type == 'Pass') and (x.outcomeType[row] == 'Successful'):
        x.receivedFrom[row+1] = x.playerId[row]
for row, type in enumerate(x.type):
    if (type == 'Pass') and (x.outcomeType[row] == 'Successful'):
        x.deliveredTo[row] = x.playerId[row+1]

In [ ]:
x= x[x.type !='Good Skill']
x['isOwnGoal'] = x['isOwnGoal'].fillna(0).astype('bool')
x['isGoodSkill'] = x['isGoodSkill'].fillna(0).astype('bool')
x['isChipped'] = x['isChipped'].fillna(0).astype('bool')
x['isBigChance'] = x['isBigChance'].fillna(0).astype('bool')
x['isKeyPass'] = x['isKeyPass'].fillna(0).astype('bool')
x['isAssist'] = x['isAssist'].fillna(0).astype('bool')
x['isLongBall'] = x['isLongBall'].fillna(0).astype('bool')
x['url'] = x['url'].fillna(0).astype('string')



In [ ]:
x